In [ ]:
%load_ext nb_black

In [1]:
%load_ext autoreload
%autoreload 2

In [184]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
import iexfinance.stocks as stocks
import tools
import os

TOKEN = 'pk_0ba3095274d146efaaa32c3536b75320'
PROJECT_PATH = '\\'.join(os.getcwd().split('\\')[:-1])
DATA_PATH = os.path.join(PROJECT_PATH, 'data')

ModuleNotFoundError: No module named 'tools'

In [54]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers = table['Symbol'].tolist()

In [88]:
def get_info(tickers):
    return stocks.Stock(tickers, token=TOKEN).get_company()

def process_info(info):
    info['industry'] = info['industry'].str[:80] + '...'
    info['industry'] = info['industry'].replace('...', 'UNKNOWN')
    return info

info = pd.concat([get_info(tickers[i:i+100]) for i in range(0, len(tickers), 100)])
info = process_info(info)
info.to_pickle(os.path.join(DATA_PATH, 'sp500_info.p'))

In [138]:
pd.Series(info['tags'].sum()).value_counts()

Manufacturing                                        195
Finance                                               99
Finance and Insurance                                 93
Information                                           53
Electronic Technology                                 52
                                                    ... 
Wireless Telecommunications                            1
Offices of Other Holding Companies                     1
Animal (except Poultry) Slaughtering                   1
Engineering Services                                   1
Wood Kitchen Cabinet and Countertop Manufacturing      1
Length: 330, dtype: int64

In [177]:
info[info['tags'].apply(','.join).str.contains('Financ')];

In [128]:
companies = info[info['tags'].apply(','.join).str.contains('Financ')]

In [143]:
start = '2012-01-01'
tickers = companies['symbol'].tolist()[:5]
data = {}
for ticker in tickers:
    ticker_object = yf.Ticker(ticker)
    data[ticker] = ticker_object.history(start=start)['Close']

In [144]:
pd.DataFrame(data)

,AFL,ARE,ALL,AXP,AIG
Date,,,,,
2012-01-03,17.976095,51.734520,23.181524,42.357841,20.442055
2012-01-04,17.851936,50.734722,23.022797,42.384182,20.314663
2012-01-05,17.968086,51.584190,23.173172,42.876202,20.306171
2012-01-06,17.719757,51.268440,23.407072,42.410557,19.991934
2012-01-09,17.315210,51.456383,23.507317,42.515980,20.382608
...,...,...,...,...,...
2020-12-24,43.840000,173.414108,107.190002,117.349998,37.369999
2020-12-28,43.919998,174.775574,107.470001,118.360001,37.070000
2020-12-29,43.750000,174.070007,107.519997,118.139999,37.160000


In [167]:
function_words = ['ANALYTICS', 'DATA', 'QUANTITATIVE', 'STATISTICAL']
level_words = ['JUNIOR', 'JR', 'SENIOR', 'SR', 'LEAD', 'STAFF', 'MANAGER', 'DIRECTOR', 'CHIEF']
combo_words = ['+'.join([l, f]) for l in level_words for f in function_words]
search_terms = function_words + combo_words

In [ ]:
tables = []
for term in search_terms[:]:
    print(term, end=' ')
    for year in range(2012, 2021):
        table = pd.read_html(f'https://h1bdata.info/index.php?em=&job={term}&city=&year={year}')[0]
        tables.append(table)

In [179]:
filings = pd.concat(tables)
filings.to_pickle(os.path.join(DATA_PATH, 'h1b_filings.p'))

In [180]:
filings

,EMPLOYER,JOB TITLE,BASE SALARY,LOCATION,SUBMIT DATE,START DATE,CASE STATUS
0,GSD&M IDEA CITY LLC,ANALYTICS MANAGER,73861,"AUSTIN, TX",03/09/2012,03/12/2012,WITHDRAWN
0,IMMUNOGEN INC,ANALYTICAL ASSOCIATE III,64625,"WALTHAM, MA",01/23/2013,03/15/2013,WITHDRAWN
1,PFIZER INC,ANALYTICAL BIOCHEMIST,79200,"SAN DIEGO, CA",03/01/2013,08/28/2013,WITHDRAWN
2,MEDIACOM WORLDWIDE INC,ANALYTICS DIRECTOR,145000,"NEW YORK, NY",05/22/2013,08/01/2013,WITHDRAWN
3,UBER TECHNOLOGIES INC,ANALYTICS ENGINEER,80000,"SAN FRANCISCO, CA",03/21/2013,09/20/2013,WITHDRAWN
...,...,...,...,...,...,...,...
0,GTS SECURITIES LLC,HEAD OF QUANTITATIVE STRATEGIES,375000,"NEW YORK, NY",08/23/2018,10/01/2018,CERTIFIED
0,VETA INVESTMENT PARTNERS LLC,HEAD OF QUANTITATIVE RESEARCH & ANALYTICS,58219,"TOPEKA, KS",09/13/2019,09/18/2019,CERTIFIED
0,CAPITAL TECHNOLOGY SOLUTIONS INC,HEAD OF QUANTITATIVE RESEARCH,200000,"NEW YORK, NY",04/03/2020,10/01/2020,CERTIFIED
1,LAKESIDE TRADING LLC,HEAD OF QUANTITATIVE RESEARCH,220000,"CHICAGO, IL",04/17/2020,05/01/2020,CERTIFIED
